<center><h1>Data engineering project</h1></center>

# Part 3 - ETL


In this final part, we'll:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json` prapared from part-1
*   Transform the market cap currency using the `exchange rate data` prapared from part-2
*   Load the transformed data into a seperate CSV


In [26]:
#Again, uncomment the needed line and run the cell
#!pip install glob
#!pip install pandas
#!pip install datetime

## Imports

In [27]:
import pandas as pd
from datetime import datetime

## Extract


### JSON Extract Function

This function will extract JSON files.


In [28]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

we'll define the extract function that finds JSON file `bank_market_cap.json` and calls the function created above to extract data it then store the data in a `pandas` dataframe.


In [29]:
def extract():
    # create an empty data frame to hold extracted data
    extracted_data = pd.DataFrame(columns=['Name','Market Cap (US$ Billion)']) 
    extracted_data = extracted_data.append(extract_from_json('bank_market_cap.json'), ignore_index=True)
    return extracted_data

Right now, we'll load the file <code>exchange_rates.csv</code> based on `EUR` prepared from part-2 as a dataframe and find the exchange rate for USD with the symbol <code>USD</code>, store it in the variable  <code>exchange_rate</code>.

In [30]:
df=pd.read_csv('exchange_rates.csv',index_col=0)
exchange_rate = df.loc['USD']['rates']
exchange_rate

1.131945

## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file, we'll find the exchange rate of EUR to USD and then write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to EUR
2.  Rounds the `Market Cap` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (EUR€ Billion)`


In [31]:
def transform(data, r):
    data['Market Cap (US$ Billion)'] = round((data['Market Cap (US$ Billion)'] / r),3)
    data.rename(columns={'Market Cap (US$ Billion)':'Market Cap (EUR€ Billion)'},inplace=True)
    return data

## Load

we'll create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`.


In [32]:
target_file = 'bank_market_cap_gbp.csv'
def load(target_file,data_to_load):
    data_to_load.to_csv(target_file,index=False) 

## Logging Function


In [33]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running the ETL Process


In [34]:
log('ETL Job Started')
log('Extract phase Started')

### Extract


In [35]:
MarketCap = extract()
MarketCap.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,488.470
1,Bank of America,379.250
2,Industrial and Commercial Bank of China,246.500
3,Wells Fargo,308.013
4,China Construction Bank,257.399


In [36]:
log('Extract phase Ended')

### Transform


In [37]:
log("Transform phase Started")

In [38]:
transformed_data = transform(MarketCap,exchange_rate)
transformed_data.head()

,Name,Market Cap (EUR€ Billion)
0,JPMorgan Chase,431.532
1,Bank of America,335.043
2,Industrial and Commercial Bank of China,217.767
3,Wells Fargo,272.110
4,China Construction Bank,227.395


In [39]:
log("Transform phase Ended")

### Load


In [40]:
log("Load phase Started")

In [41]:
load(target_file,transformed_data)

In [42]:
log("Load phase Ended")

In [43]:
!cat logfile.txt

2021-Dec-27-04:21:13,ETL Job Started
2021-Dec-27-04:21:13,Extract phase Started
2021-Dec-27-04:21:13,Extract phase Ended
2021-Dec-27-04:21:13,Transform phase Started
2021-Dec-27-04:21:13,Transform phase Ended
2021-Dec-27-04:21:13,Load phase Started
2021-Dec-27-04:21:13,Load phase Ended
